In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from pandas import DataFrame
from datetime import datetime
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten, Reshape, Activation
from keras.layers import Conv1D, MaxPooling1D ,AveragePooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras import backend as K
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.optimizers import Adam, RMSprop
from keras.layers import Input, Dense, Conv1D, MaxPooling1D
from keras.layers import Flatten, Concatenate, Dropout, Reshape, BatchNormalization
from keras.models import Model

# input file path
workdir = "E://Ronny_TF//200914_reupload//Dataset//Scenario4_EC-11K"

Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16570483069899839504
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10011936973
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13953218691367664510
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [3]:
# input and index file
dataset = np.load(workdir + "//200914_EC11K.npz")
ss0 = np.load(workdir + '//200914_EC11K_r0_9_1.npz')

In [4]:
# input and index file assign
x = dataset['x']
y = dataset['y']
# y_linear = dataset['y_lnIC50']
ss0_train = ss0['train']
ss0_test = ss0['test']

In [5]:
# training and test dataset assign
training_image_array, training_label_array = x[ss0_train], y[ss0_train]
test_image_array, test_label_array = x[ss0_test], y[ss0_test]


In [6]:
# checking the size of dataset
x.shape

(11360, 18988)

In [7]:
# checking the size of training set
training_image_array.shape

(10224, 18988)

In [8]:
# hyperparameters of this model
num_classes = 1
learning_rate = 0.0002
training_epochs = 150
batch_size = 100


In [9]:
train_X, train_y, test_X, test_y = training_image_array, training_label_array, test_image_array, test_label_array




In [10]:
shape_value = train_X.shape[1]
print(shape_value)

18988


In [11]:
# data shaping for CNN model
train_X_cell = train_X[:,:shape_value]
train_X_drug =train_X[:,shape_value:]
test_X_cell = test_X[:,:shape_value]
test_X_drug = test_X[:,shape_value:]

In [12]:
train_X_cell.shape

(10224, 18988)

In [13]:
train_X_cell = train_X_cell.reshape(train_X_cell.shape[0], train_X_cell.shape[1], 1)
train_X_drug = train_X_drug.reshape(train_X_drug.shape[0], train_X_drug.shape[1], 1)
test_X_cell = test_X_cell.reshape(test_X_cell.shape[0], test_X_cell.shape[1], 1)
test_X_drug = test_X_drug.reshape(test_X_drug.shape[0], test_X_drug.shape[1], 1)
#input_shape = (img_rows, img_cols, 1)

In [14]:
print(train_X_cell.shape,train_X_drug.shape, train_y.shape, test_X_cell.shape,test_X_drug.shape, test_y.shape)

(10224, 18988, 1) (10224, 0, 1) (10224,) (1136, 18988, 1) (1136, 0, 1) (1136,)


In [15]:
train_X_cell.shape

(10224, 18988, 1)

In [16]:
# data typing and the number of samples
train_X_cell = train_X_cell.astype('float32')
test_X_cell = test_X_cell.astype('float32')
train_X_drug = train_X_drug.astype('float32')
test_X_drug = test_X_drug.astype('float32')
# train_X /= 255
# test_X /= 255
print('train_X shape:', train_X.shape)
print(train_X.shape[0], 'train samples')
print(test_X.shape[0], 'test samples')

train_X shape: (10224, 18988)
10224 train samples
1136 test samples


In [17]:
# model architecture
with K.tf.device('/GPU:0'):
    input_cell = Input(shape=(train_X_cell.shape[1], train_X_cell.shape[2]), name='cell_input')
#     input_drug = Input(shape=(train_X_drug.shape[1], train_X_drug.shape[2]), name='drug_input')

    conv_cell_line_1 = Conv1D(filters=50, kernel_size=700, strides=5, activation='tanh')(input_cell)
    maxpool_cell_line_1 = MaxPooling1D(pool_size=5)(conv_cell_line_1)
    conv_cell_line_2 = Conv1D(filters=30, kernel_size=5, strides=2, activation='relu')(maxpool_cell_line_1)
    maxpool_cell_line_2 = MaxPooling1D(pool_size=10)(conv_cell_line_2)
    flatten_cell_line = Flatten()(maxpool_cell_line_2)
    dense_cell_line = Dense(100, activation='relu')(flatten_cell_line)
    dropout_cell_line = Dropout(0.1)(dense_cell_line)

#     conv_drug_1 = Conv1D(filters=50, kernel_size=200, strides=3, activation='tanh')(input_drug)
#     maxpool_drug_1 = MaxPooling1D(pool_size=5)(conv_drug_1)
#     conv_drug_2 = Conv1D(filters=30, kernel_size=50, strides=5, activation='relu')(maxpool_drug_1)
#     maxpool_drug_2 = MaxPooling1D(pool_size=10)(conv_drug_2)
#     flatten_drug = Flatten()(maxpool_drug_2)
#     dense_drug = Dense(100, activation='relu')(flatten_drug)
#     dropout_drug = Dropout(0.1)(dense_drug)

#     concatenate = Concatenate()([dropout_cell_line, dropout_drug])
    dense_1 = Dense(300, activation='tanh')(dropout_cell_line)
    dropout_1 = Dropout(0.1)(dense_1)
    reshape_1 = Reshape((300,1))(dropout_1)
    conv_1 = Conv1D(filters=30, kernel_size=150, strides=1, activation='relu')(reshape_1)
    maxpool_1 = MaxPooling1D(pool_size=2)(conv_1)
    conv_2 = Conv1D(filters=10, kernel_size=5, strides=1, activation='relu')(maxpool_1)
    maxpool_2 = MaxPooling1D(pool_size=3)(conv_2)
    conv_3 = Conv1D(filters=5, kernel_size=5, strides=1, activation='relu')(maxpool_2)
    maxpool_3 = MaxPooling1D(pool_size=3)(conv_3)
    dropout_2 = Dropout(0.1)(maxpool_3)
    flatten = Flatten()(dropout_2)
    dropout_3 = Dropout(0.2)(flatten)
    output = Dense(1, activation='linear', name='pred_lnIC50')(dropout_3)
    model = Model(inputs=[input_cell], outputs=output, name='MasterModel')


W0915 13:45:07.770640  3296 deprecation_wrapper.py:119] From c:\users\a\anaconda3\envs\tf_gpu_3.6\lib\site-packages\keras\backend\tensorflow_backend.py:514: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0915 13:45:07.773567  3296 deprecation_wrapper.py:119] From c:\users\a\anaconda3\envs\tf_gpu_3.6\lib\site-packages\keras\backend\tensorflow_backend.py:71: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0915 13:45:07.774544  3296 deprecation_wrapper.py:119] From c:\users\a\anaconda3\envs\tf_gpu_3.6\lib\site-packages\keras\backend\tensorflow_backend.py:4076: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0915 13:45:07.790160  3296 deprecation_wrapper.py:119] From c:\users\a\anaconda3\envs\tf_gpu_3.6\lib\site-packages\keras\backend\tensorflow_backend.py:3900: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0915 13:45:07.821393  3296 dep

In [18]:
# model architecture
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cell_input (InputLayer)      (None, 18988, 1)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 3658, 50)          35050     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 731, 50)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 364, 30)           7530      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 36, 30)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1080)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               108100    
__________

In [19]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [20]:
# model fitting
optim = Adam(lr=learning_rate)
model.compile(loss='mse',
		      optimizer=optim,
			  metrics=['mse','mae'])

StartTime8 = datetime.now()
print("StartTime :", StartTime8)
with K.tf.device('/GPU:0'):
    model_train = model.fit(train_X_cell, training_label_array, batch_size=batch_size,epochs=training_epochs,verbose=1,
                        validation_data=(test_X_cell, test_label_array))

EndTime8 = datetime.now()
print("EndTime :", EndTime8)

W0915 13:45:07.991243  3296 deprecation_wrapper.py:119] From c:\users\a\anaconda3\envs\tf_gpu_3.6\lib\site-packages\keras\optimizers.py:757: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



StartTime : 2020-09-15 13:45:08.011713


W0915 13:45:08.345503  3296 deprecation_wrapper.py:119] From c:\users\a\anaconda3\envs\tf_gpu_3.6\lib\site-packages\keras\backend\tensorflow_backend.py:983: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 10224 samples, validate on 1136 samples
Epoch 1/150
10224/10224 [==============================] - 6s 557us/step - loss: 4.1015 - mean_squared_error: 4.1015 - mean_absolute_error: 1.6091 - val_loss: 3.7248 - val_mean_squared_error: 3.7248 - val_mean_absolute_error: 1.4475
Epoch 2/150
10224/10224 [==============================] - 4s 343us/step - loss: 4.0599 - mean_squared_error: 4.0599 - mean_absolute_error: 1.5947 - val_loss: 3.7874 - val_mean_squared_error: 3.7874 - val_mean_absolute_error: 1.6047
Epoch 3/150
10224/10224 [==============================] - 3s 328us/step - loss: 4.0330 - mean_squared_error: 4.0330 - mean_absolute_error: 1.5861 - val_loss: 3.7502 - val_mean_squared_error: 3.7502 - val_mean_absolute_error: 1.5690
Epoch 4/150
10224/10224 [==============================] - 3s 314us/step - loss: 4.0235 - mean_squared_error: 4.0235 - mean_absolute_error: 1.5809 - val_loss: 3.7771 - val_mean_squared_error: 3.7771 - val_mean_absolute_error: 1.5966
Epoch 5/150
10224/1

Epoch 36/150
10224/10224 [==============================] - 3s 307us/step - loss: 3.8878 - mean_squared_error: 3.8878 - mean_absolute_error: 1.5238 - val_loss: 3.9618 - val_mean_squared_error: 3.9618 - val_mean_absolute_error: 1.6780
Epoch 37/150
10224/10224 [==============================] - 3s 311us/step - loss: 3.8955 - mean_squared_error: 3.8955 - mean_absolute_error: 1.5399 - val_loss: 3.7978 - val_mean_squared_error: 3.7978 - val_mean_absolute_error: 1.4858
Epoch 38/150
10224/10224 [==============================] - 3s 311us/step - loss: 3.8825 - mean_squared_error: 3.8825 - mean_absolute_error: 1.5249 - val_loss: 3.7699 - val_mean_squared_error: 3.7699 - val_mean_absolute_error: 1.5170
Epoch 39/150
10224/10224 [==============================] - 3s 311us/step - loss: 3.8671 - mean_squared_error: 3.8671 - mean_absolute_error: 1.5227 - val_loss: 3.8025 - val_mean_squared_error: 3.8025 - val_mean_absolute_error: 1.5683
Epoch 40/150
10224/10224 [==============================] - 3s 3

10224/10224 [==============================] - 3s 332us/step - loss: 3.8638 - mean_squared_error: 3.8638 - mean_absolute_error: 1.5146 - val_loss: 3.9349 - val_mean_squared_error: 3.9349 - val_mean_absolute_error: 1.6442
Epoch 72/150
10224/10224 [==============================] - 3s 312us/step - loss: 3.8658 - mean_squared_error: 3.8658 - mean_absolute_error: 1.5272 - val_loss: 3.8220 - val_mean_squared_error: 3.8220 - val_mean_absolute_error: 1.5304
Epoch 73/150
10224/10224 [==============================] - 3s 321us/step - loss: 3.8709 - mean_squared_error: 3.8709 - mean_absolute_error: 1.5249 - val_loss: 3.8246 - val_mean_squared_error: 3.8246 - val_mean_absolute_error: 1.5568
Epoch 74/150
10224/10224 [==============================] - 3s 309us/step - loss: 3.8555 - mean_squared_error: 3.8555 - mean_absolute_error: 1.5168 - val_loss: 3.8151 - val_mean_squared_error: 3.8151 - val_mean_absolute_error: 1.5347
Epoch 75/150
10224/10224 [==============================] - 3s 308us/step - l

10224/10224 [==============================] - 3s 308us/step - loss: 3.8401 - mean_squared_error: 3.8401 - mean_absolute_error: 1.5142 - val_loss: 3.7849 - val_mean_squared_error: 3.7849 - val_mean_absolute_error: 1.4964
Epoch 107/150
10224/10224 [==============================] - 3s 306us/step - loss: 3.8471 - mean_squared_error: 3.8471 - mean_absolute_error: 1.5175 - val_loss: 3.8028 - val_mean_squared_error: 3.8028 - val_mean_absolute_error: 1.4809
Epoch 108/150
10224/10224 [==============================] - 3s 311us/step - loss: 3.8428 - mean_squared_error: 3.8428 - mean_absolute_error: 1.5077 - val_loss: 3.8373 - val_mean_squared_error: 3.8373 - val_mean_absolute_error: 1.5654
Epoch 109/150
10224/10224 [==============================] - 3s 308us/step - loss: 3.8453 - mean_squared_error: 3.8453 - mean_absolute_error: 1.5110 - val_loss: 3.8368 - val_mean_squared_error: 3.8368 - val_mean_absolute_error: 1.5952
Epoch 110/150
10224/10224 [==============================] - 3s 308us/step

Epoch 141/150
10224/10224 [==============================] - 3s 309us/step - loss: 3.8491 - mean_squared_error: 3.8491 - mean_absolute_error: 1.5132 - val_loss: 3.8014 - val_mean_squared_error: 3.8014 - val_mean_absolute_error: 1.5160
Epoch 142/150
10224/10224 [==============================] - 3s 310us/step - loss: 3.8284 - mean_squared_error: 3.8284 - mean_absolute_error: 1.5139 - val_loss: 3.8267 - val_mean_squared_error: 3.8267 - val_mean_absolute_error: 1.4577
Epoch 143/150
10224/10224 [==============================] - 3s 308us/step - loss: 3.8389 - mean_squared_error: 3.8389 - mean_absolute_error: 1.5100 - val_loss: 3.8394 - val_mean_squared_error: 3.8394 - val_mean_absolute_error: 1.4776
Epoch 144/150
10224/10224 [==============================] - 3s 306us/step - loss: 3.8413 - mean_squared_error: 3.8413 - mean_absolute_error: 1.5138 - val_loss: 3.7959 - val_mean_squared_error: 3.7959 - val_mean_absolute_error: 1.4368
Epoch 145/150
10224/10224 [==============================] -

In [21]:
# model save
import json
from keras.models import model_from_json, load_model
workdir = "E://Ronny_TF//200914_reupload//Result//EC-11K"
# Option 1: Save Weights + Architecture
model.save_weights(workdir + '//EC11K_CNN_model_weight_fix_v3.h5')
with open(workdir + '//EC11K_CNN_model_architecture_fix_v3.json', 'w') as f:
    f.write(model.to_json())

In [22]:
# Option 2: Save/Load the Entire Model
from keras.models import load_model

# Creates a HDF5 file 'my_model.h5'
model.save(workdir + '//EC11K_CNN_model_fix_2080ti_v3.h5')

# Deletes the existing model
# del model  

# Returns a compiled model identical to the previous one
# model = load_model('my_model.h5')

In [23]:
# model evaluation
test_eval = model.evaluate([test_X_cell,test_X_drug], test_label_array, verbose=1)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 2 arrays: [array([[[-0.451841  ],
        [-0.84097344],
        [ 1.303866  ],
        ...,
        [-1.2417198 ],
        [-1.0213133 ],
        [-1.2103953 ]],

       [[-0.3259518 ],
        [ 0.44660294],
...

In [ ]:
test_eval

In [ ]:
# loss record
accuracy = model_train.history['mean_squared_error']
val_accuracy = model_train.history['val_mean_squared_error']
loss = model_train.history['loss']
val_loss = model_train.history['val_loss']

np_acc = np.array(accuracy)
np_val_acc = np.array(val_accuracy)
np_loss = np.array(loss)
np_val_loss = np.array(val_loss)

In [ ]:
# loss record
np.savetxt("EC11K_CNN_acc_cls3_fix_2080ti_v3.csv", np_acc, delimiter=",")
np.savetxt("EC11K_CNN_val_acc_cls3_fix_2080ti_v3.csv", np_val_acc, delimiter=",")
np.savetxt("EC11K_CNN_loss_cls3_fix_2080ti_v3.csv", np_loss, delimiter=",")
np.savetxt("EC11K_CNN_val_loss_cls3_fix_2080ti_v3.csv", np_val_loss, delimiter=",")

In [ ]:
import matplotlib
from matplotlib import pyplot as plt

In [ ]:
# accuracy = model_train.history['acc']
# val_accuracy = model_train.history['val_acc']
# loss = model_train.history['loss']
# val_loss = model_train.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training mse',c='red')
plt.plot(epochs, val_accuracy, 'b', label='Validation mse',c='red')
plt.title('Training and validation mse')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss',c='red')
plt.plot(epochs, val_loss, 'b', label='Validation loss',c='red')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
predicted_classes = model.predict(test_X_cell)

In [ ]:
predicted_value = predicted_classes

In [ ]:
# saving predicted ln(IC50)s and ln(IC50)s in the test set
a = pd.DataFrame(predicted_value)
b = pd.DataFrame(test_label_array)
c = pd.concat([a,b], axis=1)
c.columns=["Predicted","Test"]
c.to_csv(workdir + '//200915_EC11K_CNN_pred_and_val_result.csv', index=False)


In [ ]:
# result table
c.head()

In [ ]:
predicted_value.shape

In [ ]:
# scatter plot
plt.scatter(test_label_array,predicted_value,c='red')
plt.xlabel('test_IC50_value')
plt.ylabel('predicted_IC50_value')
plt.savefig(workdir + '//EC11K_CNN_test_scatterplot.png')

In [ ]:
# R-squared value
from sklearn.metrics import r2_score
r2_value = r2_score(b,a)
print(r2_value)

In [ ]:
# Linear regression value (slope)
from scipy.stats import linregress
linregress(b[0], a[0])

In [ ]:
# RMSE
rse = ((b[0]-a[0])**2).sum()
mse = rse / len(b)
print("Final rmse value is =",np.sqrt(mse))